In [2]:
import VAE
import torch
from torch import nn
import math

In [3]:
dloader = VAE.DataLoader(50, 25, './data/h3.6m/dataset')

Reading subject 1, action walking, subaction 1
Reading subject 1, action walking, subaction 2
Reading subject 1, action eating, subaction 1
Reading subject 1, action eating, subaction 2
Reading subject 1, action smoking, subaction 1
Reading subject 1, action smoking, subaction 2
Reading subject 1, action discussion, subaction 1
Reading subject 1, action discussion, subaction 2
Reading subject 1, action directions, subaction 1
Reading subject 1, action directions, subaction 2
Reading subject 1, action greeting, subaction 1
Reading subject 1, action greeting, subaction 2
Reading subject 1, action phoning, subaction 1
Reading subject 1, action phoning, subaction 2
Reading subject 1, action posing, subaction 1
Reading subject 1, action posing, subaction 2
Reading subject 1, action purchases, subaction 1
Reading subject 1, action purchases, subaction 2
Reading subject 1, action sitting, subaction 1
Reading subject 1, action sitting, subaction 2
Reading subject 1, action sittingdown, subacti

In [4]:
encoder_data, discriminator_data, yhat = dloader.get_train_batch(32)

In [5]:
encoder_data.shape[1:3]

(49, 54)

In [6]:
a = torch.Tensor(encoder_data)

In [7]:
encoder = VAE.Encoder(2)

In [8]:
encoder_test = encoder.forward(a)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x8064 and 2646x527)

In [3]:
dloader.train_set[(1,'walking', 1, 'even')].shape

(1738, 54)

In [12]:
a = VAE.EncoderBase(1,2,3,4)

In [1]:
import torch

In [3]:
torch.float32

torch.float32

In [11]:
for x in range(0, 10, 2):
    print(x)

0
2
4
6
8


In [12]:
import data_utils

In [18]:
filename = '{0}/S{1}/{2}_{3}.txt'.format('./data/h3.6m/dataset', 1, 'walking', 1)
action_sequence = data_utils.readCSVasFloat(filename)

In [19]:
action_sequence.shape

(3476, 99)